# CONSULTING PROJECT - CIRCADIAN TRUST

## Data PreProcessing

#### Load the Data

In [1]:
import pandas as pd

# Load the Excel file (Original as received from Circadian Trust)
file_path = r"/content/Data Bristol Uni - Spell Checked.xlsx"
df = pd.read_excel(file_path, engine='openpyxl') # Use openpyxl engine to handle encoding

# Display basic info
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7311 entries, 0 to 7310
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Site               7311 non-null   object        
 1   Gender             7311 non-null   object        
 2   Postcode           7311 non-null   object        
 3   Plan               7310 non-null   object        
 4   Age                7311 non-null   int64         
 5   Signing/Join Date  7311 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 342.8+ KB


,Site,Gender,Postcode,Plan,Age,Signing/Join Date
0,LG,F,BS16 6HQ,12 Month Lifestyle On Referral Membership\t,78,2015-07-03
1,LG,M,BS30 8GT,12 Month Total Lifestyle Corporate Membership,37,2015-07-03
2,LG,F,BS15 9UW,12 Month Total Lifestyle Older Adult Membership,72,2015-07-06
3,LG,F,BS15 3PH,12 Month Total Lifestyle Corporate Membership,51,2015-07-09
4,LG,F,BS5 8JE,12 Month Total Lifestyle Corporate Membership,57,2015-07-09


#### Checking columns "Site" and "Gender"

In [2]:
# Check unique site names
print(df["Site"].unique())

['LG' 'KW' 'TH']


In [3]:
site_mapping = {
        'TH': 'Thornbury',
        'LG': 'Longwell Green',
        'KW': 'Kingswood'
    }
df['Center'] = df['Site'].map(site_mapping)

In [4]:
df.head(10)

,Site,Gender,Postcode,Plan,Age,Signing/Join Date,Center
0,LG,F,BS16 6HQ,12 Month Lifestyle On Referral Membership\t,78,2015-07-03,Longwell Green
1,LG,M,BS30 8GT,12 Month Total Lifestyle Corporate Membership,37,2015-07-03,Longwell Green
2,LG,F,BS15 9UW,12 Month Total Lifestyle Older Adult Membership,72,2015-07-06,Longwell Green
3,LG,F,BS15 3PH,12 Month Total Lifestyle Corporate Membership,51,2015-07-09,Longwell Green
4,LG,F,BS5 8JE,12 Month Total Lifestyle Corporate Membership,57,2015-07-09,Longwell Green
5,LG,F,BS5 8JE,12 Month Total Lifestyle Adult Membership,57,2015-07-11,Longwell Green
6,LG,F,BS15 8PB,12 Month Total Lifestyle Adult Membership,50,2015-07-11,Longwell Green
7,LG,F,BS15 4JE,12 Month Lifestyle Concession Membership,67,2015-07-12,Longwell Green
8,LG,M,BS15 9QS,12 Month Total Lifestyle Adult Couple Membership,43,2015-07-15,Longwell Green
9,LG,M,BS30 9BG,12 Month Lifestyle On Referral Membership,66,2015-07-15,Longwell Green


In [5]:
# Check unique values in the Gender column
print(df["Gender"].unique())

# Count occurrences of each gender category
print(df["Gender"].value_counts())

['F' 'M']
Gender
F    4473
M    2838
Name: count, dtype: int64


#### "Postcode" Correction

In [6]:
import pandas as pd
import re

# Define a regex pattern for valid UK postcodes
postcode_pattern = r"^[A-Z]{1,2}[0-9R][0-9A-Z]?\s?[0-9][A-Z]{2}$"

# Function to clean and format UK postcodes
def clean_uk_postcode(postcode):
    if not isinstance(postcode, str):
        return postcode

    # Remove all spaces and convert to uppercase
    cleaned = postcode.strip().upper()

    # Extract alphanumeric characters only
    cleaned = re.sub(r'[^A-Z0-9]', '', cleaned)

    # Check if we have a reasonable length for a UK postcode (5-7 characters)
    if 5 <= len(cleaned) <= 7:
        # Format with a space in the correct position
        # UK postcodes have the last 3 characters as the inward code
        outward = cleaned[:-3]
        inward = cleaned[-3:]
        formatted = f"{outward} {inward}"

        # Verify the formatted postcode against the pattern
        if re.match(postcode_pattern, formatted):
            return formatted

    # If we can't format it properly, return the original with a flag
    return f"INVALID:{postcode}"

# Apply the cleaning function to the Postcode column
df['Original_Postcode'] = df['Postcode'].copy()  # Keep original for reference
df['Postcode'] = df['Postcode'].apply(clean_uk_postcode)

# Identify postcodes that couldn't be corrected
still_invalid = df[df['Postcode'].str.startswith('INVALID:', na=False)]

# Identify corrected postcodes
corrected = df[
    (~df['Postcode'].str.startswith('INVALID:', na=False)) &
    (df['Postcode'] != df['Original_Postcode'])
]

# Print results
print(f"Total postcodes: {len(df)}")
print(f"Successfully formatted postcodes: {len(corrected)}")
print(f"Postcodes that could not be corrected: {len(still_invalid)}")

# Print examples of corrections
if len(corrected) > 0:
    print("\nSample of corrected postcodes:")
    print(corrected[['Original_Postcode', 'Postcode']].head(10))

# Print examples of still invalid postcodes
if len(still_invalid) > 0:
    print("\nSample of postcodes that could not be corrected:")
    print(still_invalid[['Original_Postcode', 'Postcode']].head(10))

# Print the first few rows of the DataFrame with cleaned postcodes
print("\nSample of data with cleaned postcodes:")
print(df[['Site', 'Gender', 'Original_Postcode', 'Postcode', 'Plan']].head(10))

# Print summary statistics
print("\nPostcode correction summary:")
correction_status = df['Postcode'].str.startswith('INVALID:', na=False)
print(f"Valid postcodes: {(~correction_status).sum()}")
print(f"Invalid postcodes: {correction_status.sum()}")

Total postcodes: 7311
Successfully formatted postcodes: 360
Postcodes that could not be corrected: 13

Sample of corrected postcodes:
    Original_Postcode  Postcode
57            BS307DP  BS30 7DP
61             BS58ST   BS5 8ST
66            BS311WZ  BS31 1WZ
117           SN101FE  SN10 1FE
204           BS167AS  BS16 7AS
244           BS305RA  BS30 5RA
326           BS163TU  BS16 3TU
681           BS159GR  BS15 9GR
872           BS306LQ  BS30 6LQ
883           BS159NG  BS15 9NG

Sample of postcodes that could not be corrected:
     Original_Postcode          Postcode
998           BS15 90W  INVALID:BS15 90W
1039          BS15 90W  INVALID:BS15 90W
2831          BS15 1 N  INVALID:BS15 1 N
2991           BS14ORU   INVALID:BS14ORU
3109              BS15      INVALID:BS15
3556          BS16;R44  INVALID:BS16;R44
4377                B3        INVALID:B3
4397          BS35 IBS  INVALID:BS35 IBS
4971           BS35 1J   INVALID:BS35 1J
5451           BS35 2Y   INVALID:BS35 2Y

Sample of da

#### "Plan" Correction

In [7]:
# Display unique values in the Plan column
unique_plans = df["Plan"].unique()

# Print unique values
print("Unique Entries in Plan Column:\n")
for plan in unique_plans:
    print(plan)

Unique Entries in Plan Column:

12 Month Lifestyle On Referral Membership	
12 Month Total Lifestyle Corporate Membership 
12 Month Total Lifestyle Older Adult Membership
12 Month Total Lifestyle Adult Membership 
12 Month Lifestyle Concession Membership 
12 Month Total Lifestyle Adult Couple Membership 
12 Month Lifestyle On Referral Membership 
12 Month Total Lifestyle Concession Membership 
12 Month Total Lifestyle Corporate Membership
12 Month Lifestyle Corporate Membership 
12 Month Total Lifestyle Concession Membership
12 Month Lifestyle Older Adult Membership 
12 Month Total Lifestyle Older Adult Membership 
12 Month Total Lifestyle Junior (11-16yrs) Membership 
12  lifestyle older adult membership 
12 Month Lifestyle On Referral Membership
12 Month Total Lifestyle Student (16yrs+) Membership 
12 Month Lifeshape - NHS Referral Membership
12 Month EOP - NHS Referral Membership
12 Month Total Lifestyle Student (18yrs+) Membership 
12 Month Lifestyle Concession Membership
12 Month T

In [8]:
import re

# Step 1: Basic Cleaning Function
def basic_clean_plan(plan):
    if isinstance(plan, str):
        plan = plan.strip()  # Remove leading and trailing spaces
        plan = re.sub(r'\s+', ' ', plan)  # Remove double spaces

        # Convert to Title Case (first letter of each word capitalized)
        plan = plan.title()

    return plan

# Apply the cleaning function to the "Plan" column
df["Plan"] = df["Plan"].apply(basic_clean_plan)

# Get cleaned unique values
cleaned_plans = df["Plan"].unique()

# Step 4: Sort and display unique plans alphabetically
sorted_plans = sorted([str(plan) for plan in cleaned_plans])
print("\nAlphabetically Sorted Unique Entries:\n")
for plan in sorted_plans:
    print(plan)


Alphabetically Sorted Unique Entries:

12 Lifestyle Adult Corporate Membership
12 Lifestyle Adult Membership
12 Lifestyle Junior
12 Lifestyle Older Adult Membership
12 Month Adult Concession
12 Month Adult Lifestyle
12 Month Adult Lifestyle Adult
12 Month Adult Lifestyle Membership
12 Month Adult Membership
12 Month Corporate Lifestyle Membership
12 Month Corporate Membership
12 Month Eop - Nhs Referral Membership
12 Month Lifeshape - Nhs Referral Membership
12 Month Lifestyle Adult
12 Month Lifestyle Adult Couple Membership
12 Month Lifestyle Adult Membership
12 Month Lifestyle Adult Membership - Concession From April 2024
12 Month Lifestyle Bodytone Concession
12 Month Lifestyle Concession
12 Month Lifestyle Concession (Benefits/Health) Membership
12 Month Lifestyle Concession Membership
12 Month Lifestyle Corporate
12 Month Lifestyle Corporate Membership
12 Month Lifestyle Junior (11-17Yrs) Membership
12 Month Lifestyle Membership
12 Month Lifestyle Of Referral
12 Month Lifestyle O

In [9]:
# Replace specific variations with the standardized name
df["Plan"] = df["Plan"].replace({
    "12 Month Adult Lifestyle": "12 Month Lifestyle Adult Membership",
    "12 Month Adult Lifestyle Adult": "12 Month Lifestyle Adult Membership",
    "12 Month Adult Lifestyle Membership": "12 Month Lifestyle Adult Membership",
    "12 Month Corporate Lifestyle Membership": "12 Month Lifestyle Corporate Membership",
    "12 Month Eop - Nhs Referral Membership": "12 Month EOP - NHS Referral Membership",
    "12 Month Lifeshape - Nhs Referral Membership": "12 Month Lifeshape - NHS Referral Membership",
    "12 Month Lifestyle Adult": "12 Month Lifestyle Adult Membership",
    "12 Lifestyle Adult Corporate Membership": "12 Month Lifestyle Adult Corporate Membership",
    "12 Lifestyle Adult Membership": "12 Month Lifestyle Adult Membership",
    "12 Lifestyle Junior": "12 Month Lifestyle Junior Membership",
    "12 Lifestyle Older Adult Membership": "12 Month Lifestyle Older Adult Membership",
    "12 Month Lifestyle Concession": "12 Month Lifestyle Concession Membership",
    "12 Month Lifestyle Corporate": "12 Month Lifestyle Corporate Membership",
    "12 Month Lifestyle Of Referral": "12 Month Lifestyle On Referral",
    "12 Month Lifestyle Older Adult": "12 Month Lifestyle Older Adult Membership",
    "12 Month Lifestyle Older Person": "12 Month Lifestyle Older Adult Membership",
    "12 Month Lifestyle Older Person Membership": "12 Month Lifestyle Older Adult Membership",
    "12 Month Lifestyle On Referral Healthy Weight Management": "12 Month Lifestyle On Referral Healthy Weight Management Membership",
    "12 Month Lifestyle On Referral Healthy Weight Membership": "12 Month Lifestyle On Referral Healthy Weight Management Membership",
    "12 Month Lifestyle On Referral": "12 Month Lifestyle On Referral Membership",
    "12 Month Lifestyle Student": "12 Month Lifestyle Student Membership",
    "12 Month Lifestyle Student (18+) Membership": "12 Month Lifestyle Student (18Yrs+) Membership",
    "12 Month Lifestyle Wellbeing Studio Lor": "12 Month Lifestyle Wellbeing Studio LOR",
    "12 Month Older Lifestyle Adult": "12 Month Lifestyle Older Adult Membership",
    "12 Month Student Lifestyle Membership": "12 Month Lifestyle Student Membership",
    "3 Month Adult": "3 Month Adult Membership",
    "3 Month Adult Concessionary Membership": "3 Month Adult Concession Membership",
    "3 Month Concession": "3 Month Concession Membership",
    "3 Month Concessionary": "3 Month Concession Membership",
    "3 Month Corporate": "3 Month Corporate Membership",
    "3 Month Membership Corporate": "3 Month Corporate Membership",
    "Annual Wellbeing Stud": "Annual Wellbeing Studio",
    "Annual Wellbeing Studio Lor": "Annual Wellbeing Studio LOR",
    "Lifestyle Concession": "Lifestyle Concession Membership",
    "Lifestyle Lor": "Lifestyle On Referral Membership",
    "Lifestyle Lor Membership": "Lifestyle On Referral Membership",
    "Lifestyle On Referral": "Lifestyle On Referral Membership",
    "Lifestyle Wellbeing Concessionary": "Lifestyle Wellbeing Concession Membership",
    "Lifestyle Wellbeing Studio Concession": "Lifestyle Wellbeing Studio Concession Membership",
    "Lifestyle Wellbeing Studio Lor": "Lifestyle Wellbeing Studio LOR",
    "Month To Month Staff & Family Membership Adult": "Month To Month Staff & Family Adult Membership",
    "Month To Month Staff Family Membership Adult": "Month To Month Staff & Family Adult Membership",
    "Month To Month Staff Family Membership Child": "Month To Month Staff & Family Membership Child",
    "Month To Month Total Flexible Adult Discount (Benefits/Health)": "Month To Month Total Flexible Adult Discount (Benefits/Health) Membership",
    "Month To Month Total Flexible Lor Healthy Weight Membership": "Month To Month Total Flexible LOR Healthy Weight Membership",
    "Total Flexible Adult Disc": "Total Flexible Adult Discount",
    "Total Flexible Eop": "Total Flexible EOP",
})

# Print unique entries again to verify
# print("\nUpdated Unique Entries in Plan Column:\n")
# print(df["Plan"].unique())

# Sort and print unique membership plans alphabetically (handling mixed types)
sorted_plans = sorted(df["Plan"].dropna().astype(str).unique())  # Convert all to strings before sorting

print("\nAlphabetically Sorted Unique Entries in Plan Column:\n")
for plan in sorted_plans:
    print(plan)


Alphabetically Sorted Unique Entries in Plan Column:

12 Month Adult Concession
12 Month Adult Membership
12 Month Corporate Membership
12 Month EOP - NHS Referral Membership
12 Month Lifeshape - NHS Referral Membership
12 Month Lifestyle Adult Corporate Membership
12 Month Lifestyle Adult Couple Membership
12 Month Lifestyle Adult Membership
12 Month Lifestyle Adult Membership - Concession From April 2024
12 Month Lifestyle Bodytone Concession
12 Month Lifestyle Concession (Benefits/Health) Membership
12 Month Lifestyle Concession Membership
12 Month Lifestyle Corporate Membership
12 Month Lifestyle Junior (11-17Yrs) Membership
12 Month Lifestyle Junior Membership
12 Month Lifestyle Membership
12 Month Lifestyle Older Adult 66+ Membership
12 Month Lifestyle Older Adult Membership
12 Month Lifestyle On Referral
12 Month Lifestyle On Referral Healthy Weight Management Membership
12 Month Lifestyle On Referral Membership
12 Month Lifestyle On Referral Membership Including Bodytone And B

In [10]:
# Assuming df is your original DataFrame with 'Plan' column
# Make a copy of the original Plan column
df['Original_Plan'] = df['Plan'].copy()

# Function to clean and standardize plan strings
def clean_plan_string(plan_str):
    if isinstance(plan_str, str):
        # Basic cleaning
        plan_str = ' '.join(plan_str.split())
        # Standardize month capitalization
        plan_str = plan_str.replace('month', 'Month').replace('MONTH', 'Month')
        plan_str = plan_str.replace('Monht', 'Month').replace('Mnth', 'Month')
    return plan_str

# Apply cleaning to Plan column
df['Plan'] = df['Plan'].apply(clean_plan_string)

# Create the Membership_Type column
df['Membership_Type'] = 'Other Membership'  # Default value

# Define the patterns and categorize
# It's important to use case-insensitive matching
patterns = {
    '12 Month': r'^12\s*Month',
    '3 Month': r'^3\s*Month',
    'Month to Month': r'^Month\s*to\s*Month'
}

# Apply the patterns
for membership_type, pattern in patterns.items():
    mask = df['Plan'].str.contains(pattern, case=False, regex=True, na=False)
    df.loc[mask, 'Membership_Type'] = membership_type

# Now create the category columns correctly - this is where the fix is needed
# Initialize all category columns with None values
df['12 Month'] = None
df['3 Month'] = None
df['Month to Month'] = None
df['Other Membership'] = None

# Populate category columns based on Membership_Type
for membership_type in ['12 Month', '3 Month', 'Month to Month', 'Other Membership']:
    # Create mask for this membership type
    mask = df['Membership_Type'] == membership_type
    # Copy the plan description to the appropriate column
    df.loc[mask, membership_type] = df.loc[mask, 'Plan']

# Export to Excel
output_file = 'fixed_membership_data.xlsx'
df.to_excel(output_file, index=False)
print(f"Fixed data saved to {output_file}")

# Display summary
print("\nMembership Type Distribution:")
print(df['Membership_Type'].value_counts())

# Display sample rows to verify
print("\nSample of processed data:")
cols_to_display = ['Site', 'Gender', 'Age', 'Plan', 'Membership_Type',
                  '12 Month', '3 Month', 'Month to Month', 'Other Membership']
print(df[cols_to_display].head(10))

# Additional verification
for membership_type in ['12 Month', '3 Month', 'Month to Month', 'Other Membership']:
    non_null_count = df[membership_type].notna().sum()
    type_count = (df['Membership_Type'] == membership_type).sum()
    print(f"\n{membership_type} verification:")
    print(f"  Records with this membership type: {type_count}")
    print(f"  Non-null values in column: {non_null_count}")
    if type_count != non_null_count:
        print(f"  WARNING: Mismatch detected!")

Fixed data saved to fixed_membership_data.xlsx

Membership Type Distribution:
Membership_Type
12 Month            5442
3 Month             1191
Month to Month       376
Other Membership     302
Name: count, dtype: int64

Sample of processed data:
  Site Gender  Age                                              Plan  \
0   LG      F   78         12 Month Lifestyle On Referral Membership   
1   LG      M   37     12 Month Total Lifestyle Corporate Membership   
2   LG      F   72   12 Month Total Lifestyle Older Adult Membership   
3   LG      F   51     12 Month Total Lifestyle Corporate Membership   
4   LG      F   57     12 Month Total Lifestyle Corporate Membership   
5   LG      F   57         12 Month Total Lifestyle Adult Membership   
6   LG      F   50         12 Month Total Lifestyle Adult Membership   
7   LG      F   67          12 Month Lifestyle Concession Membership   
8   LG      M   43  12 Month Total Lifestyle Adult Couple Membership   
9   LG      M   66         12 Mon

In [11]:
# Assuming df is your original DataFrame with the issue of "None" values
# First, let's properly process the data

def process_membership_data(df):
    """
    Process the membership data and fix the None values issue
    """
    # Make a copy to avoid modifying the original
    processed_df = df.copy()

    # Clean plan strings
    def clean_plan(plan):
        if isinstance(plan, str):
            return plan.strip()
        return plan

    processed_df['Plan'] = processed_df['Plan'].apply(clean_plan)

    # Create a consistent Membership_Type column
    processed_df['Membership_Type'] = 'Other'  # Default

    # Apply pattern matching for each type
    mask_12_month = processed_df['Plan'].str.contains(r'^12\s*Month', case=False, na=False)
    processed_df.loc[mask_12_month, 'Membership_Type'] = '12 Month'

    mask_3_month = processed_df['Plan'].str.contains(r'^3\s*Month', case=False, na=False)
    processed_df.loc[mask_3_month, 'Membership_Type'] = '3 Month'

    mask_month_to_month = processed_df['Plan'].str.contains(r'^Month\s*to\s*Month', case=False, na=False)
    processed_df.loc[mask_month_to_month, 'Membership_Type'] = 'Month to Month'

    # Extract membership sub-type (Adult, Concession, etc.)
    def extract_subtype(plan):
        if not isinstance(plan, str):
            return 'Unknown'

        # Check for common subtypes
        subtypes = ['Adult', 'Concession', 'Student', 'Junior', 'Child', 'Corporate',
                   'Couple', 'Family', 'Older Adult', 'Staff', 'Referral', 'Lifestyle']

        for subtype in subtypes:
            if subtype.lower() in plan.lower():
                return subtype

        return 'Standard'

    processed_df['Membership_Subtype'] = processed_df['Plan'].apply(extract_subtype)

    # Map site codes to full names
    site_mapping = {
        'TH': 'Thornbury',
        'LG': 'Longwell Green',
        'KW': 'Kingswood'
    }
    processed_df['Center'] = processed_df['Site'].map(site_mapping)

    # Convert Signing/Join Date to datetime if it exists
    if 'Signing/Join Date' in processed_df.columns:
        processed_df['Signing/Join Date'] = pd.to_datetime(
            processed_df['Signing/Join Date'],
            format='%d-%m-%Y',
            errors='coerce'
        )

        # Add joining month and year columns
        processed_df['Join_Month'] = processed_df['Signing/Join Date'].dt.month
        processed_df['Join_Year'] = processed_df['Signing/Join Date'].dt.year
        processed_df['Join_MonthYear'] = processed_df['Signing/Join Date'].dt.strftime('%Y-%m')

    return processed_df

#### Checking columns "Age" and "Signing/Join Date"


In [12]:
# Check for any unrealistic ages (e.g., below 10 or above 100)
age_outliers = df[(df["Age"] < 10) | (df["Age"] > 100)]
print("Unrealistic Age Values:\n", age_outliers)

# Print basic statistics of Age
print("\nAge Summary Statistics:")
print(df["Age"].describe())

# Verify Joining Date format
print("\nData Type of Joining Date Column:", df["Signing/Join Date"].dtype)

# Check for future dates (e.g., greater than today's date)
from datetime import datetime
today_date = datetime.today().date()
future_dates = df[df["Signing/Join Date"] > pd.Timestamp(today_date)]
print("\nFuture Dates Found:\n", future_dates)

# Find the earliest and latest joining dates
earliest_date = df["Signing/Join Date"].min()
latest_date = df["Signing/Join Date"].max()
print(f"\nEarliest Joining Date: {df['Signing/Join Date'].min().date()}")
print(f"Latest Joining Date: {df['Signing/Join Date'].max().date()}")

Unrealistic Age Values:
      Site Gender  Postcode                         Plan  Age  \
5486   TH      M  BS35 3UE     Total Flexible Corporate  125   
5489   TH      F  BS35 4AE         Total Flexible Adult  125   
5495   TH      F  BS35 2JT  Total Flexible Older Person  125   

     Signing/Join Date     Center Original_Postcode  \
5486        2018-10-30  Thornbury          BS35 3UE   
5489        2018-11-14  Thornbury          BS35 4AE   
5495        2018-11-14  Thornbury          BS35 2JT   

                    Original_Plan   Membership_Type 12 Month 3 Month  \
5486     Total Flexible Corporate  Other Membership     None    None   
5489         Total Flexible Adult  Other Membership     None    None   
5495  Total Flexible Older Person  Other Membership     None    None   

     Month to Month             Other Membership  
5486           None     Total Flexible Corporate  
5489           None         Total Flexible Adult  
5495           None  Total Flexible Older Person  

Age

#### Exporting Cleaned Data to EXCEL File


In [13]:
# Export cleaned dataset to an Excel file
output_path_xlsx = "/content/Circadian_Trust_Cleaned_Data.xlsx"
df.to_excel(output_path_xlsx, index=False)

# Provide download link
print("Data successfully exported to Excel!")
print(f"Download link: {output_path_xlsx}")

Data successfully exported to Excel!
Download link: /content/Circadian_Trust_Cleaned_Data.xlsx
